In [78]:
import pandas as pd
import numpy as np
import re
import os
from sklearn.cluster import KMeans

In [79]:
#reading data from csv files
user_feat = pd.read_csv('user_features.csv')

print user_feat

spice_feat = pd.read_excel('test4.xls')

print spice_feat
#food user rating table
food_user = pd.DataFrame(np.random.randint(low=0, high=3, size=(len(spice_feat),len(user_feat))),columns=user_feat.user_guid)
food_user.index  =  spice_feat['food_id']
#print food_user.head()


    Age Sex          Location  NonVeg  user_guid
0    41   M  Punjab & Haryana       1        500
1    98   M   Jammu & Kashmir       0        501
2    68   M       South India       1        502
3    41   F       South India       0        503
4    14   F  Punjab & Haryana       1        504
5    88   F  Punjab & Haryana       2        505
6    35   M         Rajasthan       1        506
7    88   F       South India       0        507
8    92   M   Jammu & Kashmir       1        508
9    31   M  Punjab & Haryana       0        509
10   56   M       South India       0        510
11   85   M       Maharashtra       2        511
12   65   F       South India       0        512
13   42   M       South India       0        513
14   64   M         Rajasthan       1        514
15   75   M  Punjab & Haryana       1        515
16   30   F  Punjab & Haryana       2        516
17   52   F  Punjab & Haryana       0        517
18   72   F   Jammu & Kashmir       1        518
19   79   M  Punjab 

In [80]:
print food_user

user_guid  500  501  502  503  504  505  506  507  508  509 ...   590  591  \
food_id                                                     ...              
0            2    2    2    1    2    0    2    2    1    0 ...     0    0   
1            0    1    0    0    1    1    2    0    2    1 ...     1    2   
2            2    1    0    1    0    2    0    1    0    2 ...     2    2   
3            1    1    0    0    2    2    1    2    2    1 ...     1    0   
4            2    2    1    2    0    0    1    2    1    1 ...     0    2   
5            1    2    0    0    1    0    2    2    1    0 ...     2    0   
6            2    2    0    2    1    0    0    2    1    1 ...     2    2   
7            0    0    0    0    1    0    2    1    0    2 ...     1    2   
8            0    1    0    1    1    0    0    2    0    1 ...     0    0   
9            0    2    1    0    2    2    1    1    1    0 ...     1    2   
10           0    0    2    0    1    2    2    2    1    1 ... 

In [81]:
#Data Normalisation

def nor_age(age):
    
    if(age<=10):
        return 1
    elif(age>10 and age <=18):
        return 2
    elif(age>18 and age <=25):
        return 3
    elif(age>25 and age <=35):
        return 4
    elif(age>35 and age <=40):
        return 5
    elif(age>40 and age <=50):
        return 6
    elif(age>50 and age<=60):
        return 7
    else:
        return 8
    
    

s_dict = {"M":1,"F":5 }
l_dict = {'Jammu & Kashmir':2, 'Punjab & Haryana':5, 'Rajasthan':8, 'Maharashtra':10, 'South India':15}
user_feat['Age'] = user_feat['Age'].apply(nor_age)
user_feat  = user_feat.replace({"Sex":s_dict})
user_feat  = user_feat.replace({"Location":l_dict})

In [82]:
print "enter the user id"

input_user_id = input()

enter the user id
555


In [83]:
kmeans = KMeans(n_clusters = 10,random_state=0).fit(user_feat)
user_feat['label'] = kmeans.labels_
#print user_feat

user_feat['meal_size_rating'] = pd.DataFrame(np.random.randint(0,3, size = (len(user_feat),1)))
user_feat['previous_meal_size'] = pd.DataFrame(np.random.randint(0,3, size = (len(user_feat),1)))

index = np.where(user_feat['user_guid']==input_user_id)[0]
#print index[0]
user_label = user_feat.loc[index[0],'label']
#print user_label
user = user_feat.sort_values(['label','Location'],ascending=[1,0])
user = user.drop_duplicates(subset=['Location','label'])

user = user.groupby('label')['Location'].nlargest(3)
print user.head()
loc = user[user_label].tolist()
print loc

label    
0      21    15
       20    10
       14     8
1      70    15
       75    10
Name: Location, dtype: int64
[15, 10, 8]


In [84]:
print "enter veg/non-veg"

veggy = input()

print "enter user required calories of current meal"

upper_calorie = input()

#'Mains':0, 'Add-Ons':1,'Snacks':2, 'Beverages':3
print "enter the meal type(0,1,2,3)"

meal_type = input()

enter veg/non-veg
1
enter user required calories of current meal
200
enter the meal type(0,1,2,3)
0


In [90]:
previous_rating = user_feat.loc[input_user_id-500,'meal_size_rating']
previous_size = user_feat.loc[input_user_id-500,'previous_meal_size']

next_meal_size = previous_size

if(previous_rating==0 and previous_size<2):
    next_meal_size = previous_size + 1
elif (previous_rating==2 and previous_size>0):
    next_meal_size = previous_size - 1
    

if(veggy==0):
    spice2 = spice_feat[spice_feat['NonVeg']==veggy]
else:
    spice2 = spice_feat

spice2 = spice2[spice2['Meal Size']==next_meal_size]
    
#todo - ravindra is working on this part (5 day window)
s_index = food_user.index[food_user[input_user_id]==0]
#print s_index

in_index = spice2[spice2['food_id'].isin(s_index)]

meal_selected = in_index[in_index['Place in Menu']==meal_type]

new_spices2 = meal_selected[meal_selected['Calories']<upper_calorie]

new_spices = new_spices2[new_spices2['State'].isin(loc)]
new_spices = new_spices.set_index('food_id')
new_spices =  new_spices.sort_values('count',ascending=[0])
print new_spices

         State            Dish  Type  Place in Menu  Calories  count  \
food_id                                                                
177         10      Panchamrut     0              0     150.0      2   
206         15         Kolumbu     0              0     134.0      1   
223         15  Moru Moru Dosa     0              0     120.0      1   

         Meal Size  
food_id             
177              1  
206              1  
223              1  


In [52]:
#integrating randomness

from random import randint
#print(randint(0, 9))

x = randint(0,9)   #random with probability of 1/10
print x

3


In [53]:
recommend = []

#even in random,at least 2 constraints are there, calory and veg/non-veg and maybe location(to be confirmed) too

if(x==5):
    print ('random')  #to test
    index_to_append = new_spices2.index[randint(0,len(new_spices2))]
    #print random_feat.loc[index_to_append,'food_id']
    recommend.append(new_spices2.loc[index_to_append,'food_id'])
    
    
    for i in range(min(2,len(new_spices))):
        recommend.append(new_spices.index[i])
    
    if(len(recommend)==1):
        other_state_spice = spice_feat[spice_feat['Location'].isin(loc)==False]
    
        other_state_spice = other_state_spice.sort('count',ascending= [0])
        new_other_state = other_state_spice[other_state_spice.index.isin(s_index)]

        for i in range(min(2,len(new_other_state))):
            recommend.append(new_other_state.index[i])
            
else:
    print ('not random')
    for i in range(min(3,len(new_spices))):
        recommend.append(new_spices.index[i])
    
    if(len(recommend)<3):
        other_state_spice = spice_feat[spice_feat['Location'].isin(loc)==False]
    
        other_state_spice = other_state_spice.sort('count',ascending= [0])
        new_other_state = other_state_spice[other_state_spice.index.isin(s_index)]
        if(len(new_other_state)>=(3-len(recommend))):
            for i in range(3-len(recommend)):
                recommend.append(new_other_state.index[i])

    
print recommend    

not random
[163, 164, 129]
